# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [47]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [49]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
cityDataDF = pd.read_csv("output_data/cities.csv")

# Display sample data
cityDataDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,25.21,76,21,9.91,US,1738719940
1,1,cabedelo,-6.9811,-34.8339,75.87,82,100,10.96,BR,1738720191
2,2,malango,-2.9627,119.9001,82.63,53,81,3.94,ID,1738720193
3,3,port lincoln,-34.7333,135.8667,68.86,59,8,16.40,AU,1738720195
4,4,raychikhinsk,49.7838,129.4121,-7.80,92,100,11.32,RU,1738720197


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [54]:
%%capture --no-display

# Configure the map plot
humidityPlot = cityDataDF.hvplot.points(
    "Lng",
    "Lat",
    geo =True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["City", "Humidity"]
)
    

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [105]:
# Narrow down cities that fit criteria and drop any results with null values

idealCityDF = cityDataDF.loc[
    (cityDataDF["Max Temp"] > 21) & (cityDataDF["Max Temp"] < 27) & (cityDataDF["Wind Speed"] < 4.5) & (cityDataDF["Cloudiness"] == 0)
]

# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

0


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address


In [99]:
cityDataDF.dtypes

City_ID            int64
City              object
Lat              float64
Lng              float64
Max Temp         float64
Humidity           int64
Cloudiness         int64
Wind Speed       float64
Country           object
Date               int64
Hotel Name        object
Hotel Address     object
dtype: object

### Step 3: Create a new DataFrame called `hotel_df`.

In [78]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [80]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df

Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [3]:
# 'lon': -80.8431, 'lat': 35.2271 - Charlotte, NC

In [27]:
# Geoapify API Key
geoapify_key = "bb44fe9487604afe849b08ce3b0633f7"

In [23]:
# use the coordinates for Clt
latitude = 35.2271
longitude = -80.8431

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# set initial parameters
category = "accommodation.hotel"
radius = 8000 # sets the radius to search

# set additional parameters for search
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 10

# set up the parameters dictionary
parameters = {
    "categories": category,
    "limit": limit,
    "filters": filters,
    "bias": bias,
    "apiKey": geoapify_key
}

# run a request using the parameters dictionary
response = requests.get(base_url, params=parameters).json()

print(json.dumps(response, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "name": "Omni Charlotte Hotel",
                "country": "United States",
                "country_code": "us",
                "state": "North Carolina",
                "county": "Mecklenburg County",
                "city": "Charlotte",
                "postcode": "28202",
                "suburb": "Uptown",
                "street": "East Trade Street",
                "housenumber": "132",
                "lon": -80.8426236725816,
                "lat": 35.22621925,
                "state_code": "NC",
                "formatted": "Omni Charlotte Hotel, 132 East Trade Street, Charlotte, NC 28202, United States of America",
                "address_line1": "Omni Charlotte Hotel",
                "address_line2": "132 East Trade Street, Charlotte, NC 28202, United States of America",
                "categories": [
                    "accommodat

In [35]:
# print the name and address of the 1st hotel that appears:
print(response["features"][0]["properties"]["name"])
print(response["features"][0]["properties"]["address_line2"])

Omni Charlotte Hotel
132 East Trade Street, Charlotte, NC 28202, United States of America


In [68]:
cityDataDF.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
0,0,albany,42.6001,-73.9662,25.21,76,21,9.91,US,1738719940,Tru,"1651 Western Avenue, Westmere, NY 12203, Unite..."
1,1,cabedelo,-6.9811,-34.8339,75.87,82,100,10.96,BR,1738720191,N/A,N/A
2,2,malango,-2.9627,119.9001,82.63,53,81,3.94,ID,1738720193,Madarana,"Jalan Andi Mappanyuki, Buntu Barana 91832, Sou..."
3,3,port lincoln,-34.7333,135.8667,68.86,59,8,16.40,AU,1738720195,Boston Hotel,"19-21 King Street, Port Lincoln SA 5606, Austr..."
4,4,raychikhinsk,49.7838,129.4121,-7.80,92,100,11.32,RU,1738720197,N/A,N/A


In [67]:
# set initial parameters
category = "accommodation.hotel"
radius = 8000 # sets the radius to search

for index, row in cityDataDF.iterrows():
    # get the latitude and longitude for the city
    latitude = row["Lat"]
    longitude = row["Lng"]

    # set additional parameters for search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 1

    # set up the parameters dictionary
    parameters = {
        "categories": category,
        "limit": limit,
        "filters": filters,
        "bias": bias,
        "apiKey": geoapify_key
    }

    # run a request using the parameters dictionary
    response = requests.get(base_url, params=parameters).json()

    try:
        # try to get the first hotel, and add to the dataframe
        cityDataDF.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
        cityDataDF.loc[index, "Hotel Address"] = response["features"][0]["properties"]["address_line2"]
    except:
        # if no hotel is found, set the hotel name as N/A
        cityDataDF.loc[index, "Hotel Name"] = "N/A"
        cityDataDF.loc[index, "Hotel Address"] = "N/A"

cityDataDF.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
0,0,albany,42.6001,-73.9662,25.21,76,21,9.91,US,1738719940,Tru,"1651 Western Avenue, Westmere, NY 12203, Unite..."
1,1,cabedelo,-6.9811,-34.8339,75.87,82,100,10.96,BR,1738720191,N/A,N/A
2,2,malango,-2.9627,119.9001,82.63,53,81,3.94,ID,1738720193,Madarana,"Jalan Andi Mappanyuki, Buntu Barana 91832, Sou..."
3,3,port lincoln,-34.7333,135.8667,68.86,59,8,16.40,AU,1738720195,Boston Hotel,"19-21 King Street, Port Lincoln SA 5606, Austr..."
4,4,raychikhinsk,49.7838,129.4121,-7.80,92,100,11.32,RU,1738720197,N/A,N/A
5,5,invercargill,-46.4000,168.3500,76.01,55,100,1.99,NZ,1738720199,The Grand,"76 Dee Street, Avenal, Invercargill City 9810,..."
6,6,saipan,15.1355,145.7010,88.50,75,20,17.27,MP,1738720202,Chalan Kanoa Beach Hotel,"Atuhong Place, Chalan Piao, MP 96950, United S..."
7,7,hobyo,5.3505,48.5268,74.08,82,61,20.36,SO,1738720204,Iftin Hotel Hobyo,"Airport road, Hobyo, Somalia"
8,8,bethel,41.3712,-73.4140,30.56,57,24,1.99,US,1738720206,Hampton Inn Danbury,"81 Newtown Road, Danbury, CT 06810, United Sta..."
9,9,port mathurin,-19.6833,63.4167,80.60,80,3,9.48,MU,1738720208,Escale Vacances,"Johnston street, Port Mathurin, Mauritius"


In [19]:
print(cityDataDF.columns)

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


In [83]:
# filter the places that don't have a hotel
cityDataDF = cityDataDF.loc[cityDataDF["Hotel Name"] != "N/A"]
len(cityDataDF)

522

In [91]:
hotelPlot = cityDataDF.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Max Temp",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 896,
    frame_height = 504,
    hover_cols = ["City", "Max Temp", "Hotel Name", "Hotel Address"]
)

hotelPlot

C:\Users\Mica Lenzo\anaconda3\Lib\site-packages\holoviews\util\transform.py:655: RuntimeWarning: invalid value encountered in sqrt
  data = fn(*args, **kwargs)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Hotel Address)

In [38]:
print(cityDataDF.columns)

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date', 'Hotel Name', 'Hotel Address'],
      dtype='object')
